In [2]:
from datasets import load_from_disk, load_metric
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig, Trainer
import numpy as np
import pyarrow as pa
import pandas as pd

import os
import sys
sys.path.insert(0, '/zhome/a6/6/127219/Speciale/master_project')
from src.models.transformers_modeling_roberta import RobertaForSequenceClassification_fromTransformersLinear, RobertaForSequenceClassification_fromTransformers

In [4]:
# load
datadir = '/work3/s174498/sst2_dataset/'
#checkpoint = "/work3/s174498/final/Prob_linear_head/checkpoint-2500"
checkpoint = "/work3/s174498/final/Prob_original_head/checkpoint-1500"

# test data
test_dataset = load_from_disk(datadir + 'test_dataset')

# tokenizer
tokenizer = RobertaTokenizer.from_pretrained(checkpoint)
tokenizer.model_max_len=512

# model
#config = RobertaConfig.from_pretrained(checkpoint, output_attentions=True, output_hidden_states=True)
config = RobertaConfig.from_pretrained(checkpoint)
config.output_hidden_states = True
#config.output_attentions= True
model = RobertaForSequenceClassification_fromTransformers.from_pretrained(checkpoint, config=config)

In [5]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, max_length=512)

In [6]:
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /work3/s174498/sst2_dataset/test_dataset/cache-3101ebfc3efad1c4.arrow


In [7]:
trainer = Trainer(
    model=model,                        
    tokenizer=tokenizer
)

In [8]:
# Predicting with model
predictions = trainer.predict(tokenized_test)


The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification_fromTransformers.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `RobertaForSequenceClassification_fromTransformers.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1821
  Batch size = 8


In [9]:
pred_dataset_test = list(np.argmax(predictions.predictions[0], axis=-1))
true_dataset_test = predictions.label_ids

In [10]:
# compute accuracy
accuracy_metric = load_metric("accuracy")
accuracy_metric.compute(predictions=pred_dataset_test, references=true_dataset_test)

/zhome/a6/6/127219/miniconda3/envs/sent_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


{'accuracy': 0.9275123558484349}